#### linear, ReLU, sigmoid, tanh, and softmax activation functions as a class for neural networks implementation

In [1]:
import numpy as np

class LinearActivation:
    def forward(self, x):
        return x

class ReLUActivation:
    def forward(self, x):
        return np.maximum(0, x)

class SigmoidActivation:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

class TanhActivation:
    def forward(self, x):
        return np.tanh(x)

class SoftmaxActivation:
    def forward(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

# Example usage
x = np.array([-1, 0, 1])

relu = ReLUActivation()
print("ReLU:", relu.forward(x))

sigmoid = SigmoidActivation()
print("Sigmoid:", sigmoid.forward(x))

tanh = TanhActivation()
print("Tanh:", tanh.forward(x))

softmax = SoftmaxActivation()
print("Softmax:", softmax.forward(np.array([x, x+1])))


ReLU: [0 0 1]
Sigmoid: [0.26894142 0.5        0.73105858]
Tanh: [-0.76159416  0.          0.76159416]
Softmax: [[0.26894142 0.26894142 0.26894142]
 [0.73105858 0.73105858 0.73105858]]


#### class structure and forward propagation including the loss (cost) function implementation for a deep (multilayer) neural network

In [2]:
import numpy as np

# Activation Function Classes
class LinearActivation:
    def forward(self, x):
        return x

    def backward(self, x):
        return np.ones_like(x)

class ReLUActivation:
    def forward(self, x):
        return np.maximum(0, x)

    def backward(self, x):
        return np.where(x > 0, 1, 0)

class SigmoidActivation:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def backward(self, x):
        sigmoid = self.forward(x)
        return sigmoid * (1 - sigmoid)

class TanhActivation:
    def forward(self, x):
        return np.tanh(x)

    def backward(self, x):
        return 1 - np.tanh(x)**2

class SoftmaxActivation:
    def forward(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def backward(self, x):
        softmax = self.forward(x)
        return softmax * (1 - softmax)

# Layer Class
class Layer:
    def __init__(self, input_size, output_size, activation_function):
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.biases = np.zeros((1, output_size))
        self.activation_function = activation_function

    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        self.output = self.activation_function.forward(self.z)
        return self.output

# Neural Network Class
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def calculate_loss(self, y_pred, y_true):
        return np.mean((y_pred - y_true) ** 2)

# Example usage
nn = NeuralNetwork()
nn.add_layer(Layer(3, 5, ReLUActivation()))
nn.add_layer(Layer(5, 2, SigmoidActivation()))

# Dummy input data and true values for demonstration
input_data = np.random.randn(10, 3)
y_true = np.random.randn(10, 2)

# Forward propagation
output = nn.forward(input_data)

# Loss calculation
loss = nn.calculate_loss(output, y_true)
print(f"Loss: {loss}")


Loss: 1.1008788497199111


#### Backpropagation implementation for a deep (multilayer) neural network

In [3]:
import numpy as np

# Activation Function Classes (with backward methods)
class LinearActivation:
    def forward(self, x):
        return x

    def backward(self, dout):
        return dout

class ReLUActivation:
    def forward(self, x):
        return np.maximum(0, x)

    def backward(self, dout, inputs):
        return dout * (inputs > 0)

class SigmoidActivation:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def backward(self, dout, inputs):
        sigmoid = self.forward(inputs)
        return dout * sigmoid * (1 - sigmoid)

class TanhActivation:
    def forward(self, x):
        return np.tanh(x)

    def backward(self, dout, inputs):
        return dout * (1 - np.tanh(inputs)**2)

class SoftmaxActivation:
    def forward(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def backward(self, dout, inputs):
        softmax = self.forward(inputs)
        return dout * softmax * (1 - softmax)

# Layer Class (with backward method)
class Layer:
    def __init__(self, input_size, output_size, activation_function):
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.biases = np.zeros((1, output_size))
        self.activation_function = activation_function

    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        self.output = self.activation_function.forward(self.z)
        return self.output

    def backward(self, dout):
        dactivation = self.activation_function.backward(dout, self.z)
        self.dweights = np.dot(self.inputs.T, dactivation)
        self.dbiases = np.sum(dactivation, axis=0, keepdims=True)
        return np.dot(dactivation, self.weights.T)

# Neural Network Class (with update method for backpropagation)
class NeuralNetwork:
    def __init__(self, learning_rate=0.01):
        self.layers = []
        self.learning_rate = learning_rate

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def backward(self, y_pred, y_true):
        # Loss derivative (assuming MSE loss)
        dout = 2 * (y_pred - y_true) / y_true.size

        # Backpropagation
        for layer in reversed(self.layers):
            dout = layer.backward(dout)

    def update_weights(self):
        for layer in self.layers:
            layer.weights -= self.learning_rate * layer.dweights
            layer.biases -= self.learning_rate * layer.dbiases

    def train(self, x_train, y_train, epochs):
        for epoch in range(epochs):
            output = self.forward(x_train)
            self.backward(output, y_train)
            self.update_weights()
            if epoch % 100 == 0:  # Print loss every 100 epochs
                loss = np.mean((output - y_train) ** 2)
                print(f"Epoch {epoch}, Loss: {loss}")

# Example usage
nn = NeuralNetwork(learning_rate=0.01)
nn.add_layer(Layer(3, 5, ReLUActivation()))
nn.add_layer(Layer(5, 2, SigmoidActivation()))

# Dummy training data
x_train = np.random.randn(100, 3)
y_train = np.random.randn(100, 2)

# Training the network
nn.train(x_train, y_train, epochs=1000)


Epoch 0, Loss: 1.2852148925503146
Epoch 100, Loss: 1.260299178517058
Epoch 200, Loss: 1.2386443084347583
Epoch 300, Loss: 1.219975240445666
Epoch 400, Loss: 1.203876631933929
Epoch 500, Loss: 1.1900246640992325
Epoch 600, Loss: 1.178113502128264
Epoch 700, Loss: 1.1678628292997772
Epoch 800, Loss: 1.1590092790405258
Epoch 900, Loss: 1.1513091616649012
